In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [4]:
import json

In [5]:
from geopy.geocoders import Nominatim 


In [6]:
import requests
from pandas.io.json import json_normalize 

In [7]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [8]:
import folium

In [9]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [10]:
import matplotlib.pyplot as plt
import pylab as pl


In [11]:
from sklearn import linear_model
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, r2_score
import itertools

In [12]:
print('Libraries imported.')

Libraries imported.


In [13]:


address1 = 'Hanoi, Vietnam'

geolocator = Nominatim(user_agent="http")
location1 = geolocator.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address1, latitude1, longitude1))


address2 = 'Ho Chi Minh City, Vietnam'

geolocator = Nominatim(user_agent="http")
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address2, latitude2, longitude2))



The geograpical coordinate of Hanoi, Vietnam are 21.0294498, 105.8544441.
The geograpical coordinate of Ho Chi Minh City, Vietnam are 10.7758439, 106.7017555.


In [14]:
CLIENT_ID = 'RU5D3RUY5VCRWJOMQJZ1PEBEQRHC53EADGEOGSPHTD1CYWHO' # your Foursquare ID
CLIENT_SECRET = 'S5ZNGYTQYBFB1YRHFXY0B3RFFMIQ5NVQ0OUBLOTJRGJPCB03' # your Foursquare Secret
VERSION = '20200921' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius

# create URLs
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)


# create URLs
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)

print(url1, url2)

Your credentails:
CLIENT_ID: RU5D3RUY5VCRWJOMQJZ1PEBEQRHC53EADGEOGSPHTD1CYWHO
CLIENT_SECRET:S5ZNGYTQYBFB1YRHFXY0B3RFFMIQ5NVQ0OUBLOTJRGJPCB03
https://api.foursquare.com/v2/venues/explore?&client_id=RU5D3RUY5VCRWJOMQJZ1PEBEQRHC53EADGEOGSPHTD1CYWHO&client_secret=S5ZNGYTQYBFB1YRHFXY0B3RFFMIQ5NVQ0OUBLOTJRGJPCB03&v=20200921&ll=21.0294498,105.8544441&radius=10000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=RU5D3RUY5VCRWJOMQJZ1PEBEQRHC53EADGEOGSPHTD1CYWHO&client_secret=S5ZNGYTQYBFB1YRHFXY0B3RFFMIQ5NVQ0OUBLOTJRGJPCB03&v=20200921&ll=10.7758439,106.7017555&radius=10000&limit=100


In [15]:
# scrape the data from the generated URLs

results1 = requests.get(url1).json()
results1

results2 = requests.get(url2).json()
results2

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

### first city ###    
    
venues1 = results1['response']['groups'][0]['items']
nearby_venues1 = json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns1 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues1 =nearby_venues1.loc[:, filtered_columns1]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]


### second city ###

venues2 = results2['response']['groups'][0]['items']
nearby_venues2 = json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns2 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues2 =nearby_venues2.loc[:, filtered_columns2]

# filter the category for each row
nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

# clean columns
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues1.shape[0]))
print('{} venues were returned by Foursquare.'.format(nearby_venues2.shape[0]))


100 venues were returned by Foursquare.
100 venues were returned by Foursquare.


/Users/hainamnguyen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/Users/hainamnguyen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [16]:
# add locations data to the data sets of each city

nearby_venues1['city'] = 'Hanoi'
nearby_venues2['city'] = 'Ho Chi Minh City'

In [17]:
# combine the two cities into one data set

nearby_venues = nearby_venues1.copy()
nearby_venues = nearby_venues.append(nearby_venues2)

In [18]:
nearby_venues['categories'].unique()

removal_list = ['Concert Hall', 'Opera House', 'Dance Studio',
                'Performing Arts Venue', 'Art Museum', 'Park',
                'Massage Studio', 'Music Venue', 'Bookstore', 'Clothing Store',
                'Boutique', 'Furniture/Home Store', 'Jazz Club',
                'Theater', 'Optical Shop', "Men's Store", 'Rock Club',
                'Gym / Fitness Center', 'Wine Shop', 'Indie Movie Theater',
                'Chocolate Shop', 'Dessert Shop', 'Recreation Center', 
                'Plaza', 'Hotel', 'Luggage Store', 'Farmers Market', 'Gym',
                'Jewelry Store', 'Furniture / Home Store', 'Butcher', 
                'Bakery', 'Marijuana Dispensary', 'Ice Cream Shop',
                'Comic Shop', 'Bagel Shop', 'Spa', 'Liquor Store', 'Bike Shop',
                'Yoga Studio', 'Pedestrian Plaza', 'Candy Store',
                'Park', 'Bookstore', 'Candy Store',  'Jazz Club', 'Art Gallery', 
                 'Supermarket', 'Museum', 'Boutique', 'Plaza', 'Building', 'Bakery',
                 'Historic Site', 'Ice Cream Shop', ' Concert Hall', 'Pharmacy', 
                 'Market', 'Movie Theater', 'Performing Arts Venue', 'Music Venue',
                 'Theater', 'Art Museum', 'Cheese Shop', 'Opera House',
                 'Pedestrian Plaza', 'School', 'Gift Shop', 'Athletics & Sports',
                 'Shoe Repair', 'General Entertainment', 'Stationery Store',
                 'Toy / Game Store', 'Brewery', 'Hotel', 'Theater', 'Music Venue', 'Business Service',
                 'Donut Shop', 'Liquor Store', 'Beer Store',
                 'Lounge', 'Plaza', 'Health Food Store', 'Concert Hall', 
                 'Lingerie Store', 'Gym', 'Mobile Phone Shop',
                 'Chocolate Shop', 'Ice Cream Shop', 'Hostel', 'Convenience Store', 
                 'Park', 'Farmers Market', 'Cosmetics Shop', 'Piano Bar',
                 'Nightclub', 'Massage Studio', 'Comedy Club', 'Concert Hall', 'Cultural Center', 'Temple','Arts & Crafts Store', 'Pool', 'Department Store', 'Speakeasy','Fountain', 'Shopping Mall','Multiplex', 'Climbing Gym', 'Church', 'Golf Course', 'Bowling Alley', 'Travel Agency', 'Bed & Breakfast']
nearby_venues = nearby_venues[~nearby_venues['categories'].isin(removal_list)]

nearby_venues['categories'].unique()

array(['Italian Restaurant', 'Pizza Place', 'Vietnamese Restaurant',
       'Coffee Shop', 'Steakhouse', 'Sandwich Place', 'Bistro',
       'Noodle House', 'BBQ Joint', 'Hotel Bar', 'Beer Bar',
       'Vegetarian / Vegan Restaurant', 'French Restaurant',
       'Fast Food Restaurant', 'Café', 'Japanese Restaurant',
       'Beer Garden', 'Modern European Restaurant', 'Asian Restaurant',
       'Korean Restaurant', 'Indian Restaurant', 'Bar',
       'German Restaurant', 'Thai Restaurant', 'Whisky Bar',
       'Sushi Restaurant', 'Burger Joint', 'Convention Center',
       'Ramen Restaurant', 'Food Court', 'Tapas Restaurant',
       'Middle Eastern Restaurant', 'Breakfast Spot',
       'Chinese Restaurant', 'Hotpot Restaurant', 'Mexican Restaurant'],
      dtype=object)

In [19]:
# set up to pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in list(nearby_venues.id):
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)


nearby_venues['likes'] = like_list
nearby_venues.head()

[96, 246, 80, 87, 44, 74, 14, 290, 10, 15, 22, 405, 9, 26, 87, 138, 239, 50, 56, 317, 149, 69, 24, 35, 8, 93, 52, 455, 29, 13, 102, 29, 13, 12, 7, 16, 20, 10, 16, 40, 72, 25, 68, 9, 13, 14, 71, 26, 17, 7, 59, 17, 9, 8, 9, 7, 11, 20, 235, 172, 104, 32, 45, 514, 51, 44, 146, 92, 378, 106, 262, 57, 27, 43, 27, 44, 137, 39, 71, 337, 32, 83, 23, 26, 55, 45, 10, 24, 75, 7, 12, 236, 97, 131, 8, 43, 23, 11, 254, 24, 260, 36, 43, 321, 13, 183, 5, 12, 23, 29, 408, 15, 18, 9, 88, 20, 36, 9, 99, 17, 56, 12, 25, 54]


,name,categories,lat,lng,id,city,likes
3,Pizza 4P,Italian Restaurant,21.025141,105.854438,57d2b9d3498e1e677992b6b5,Hanoi,96
4,Pizza 4P's,Pizza Place,21.029746,105.848956,55deae83498e47aabe8a5e77,Hanoi,246
5,Orchid Cooking Class & Restaurant,Vietnamese Restaurant,21.033874,105.853270,50fbfd43e4b011743042d9d9,Hanoi,80
7,Cộng Càphê,Coffee Shop,21.035166,105.853395,555e9b4a498ea0e936c077a7,Hanoi,87
8,Starbucks Reserve Nhà Thờ,Coffee Shop,21.029162,105.850222,590d4a5e1108ba582673bec7,Hanoi,44


In [20]:
# this is really the raw dataset now so let us rename it something more appropriate

raw_dataset = nearby_venues
raw_dataset.head()

,name,categories,lat,lng,id,city,likes
3,Pizza 4P,Italian Restaurant,21.025141,105.854438,57d2b9d3498e1e677992b6b5,Hanoi,96
4,Pizza 4P's,Pizza Place,21.029746,105.848956,55deae83498e47aabe8a5e77,Hanoi,246
5,Orchid Cooking Class & Restaurant,Vietnamese Restaurant,21.033874,105.853270,50fbfd43e4b011743042d9d9,Hanoi,80
7,Cộng Càphê,Coffee Shop,21.035166,105.853395,555e9b4a498ea0e936c077a7,Hanoi,87
8,Starbucks Reserve Nhà Thờ,Coffee Shop,21.029162,105.850222,590d4a5e1108ba582673bec7,Hanoi,44


In [21]:
# inspecting the raw dataset shows that there may be too many different types of cuisines
raw_dataset['categories'].unique()

array(['Italian Restaurant', 'Pizza Place', 'Vietnamese Restaurant',
       'Coffee Shop', 'Steakhouse', 'Sandwich Place', 'Bistro',
       'Noodle House', 'BBQ Joint', 'Hotel Bar', 'Beer Bar',
       'Vegetarian / Vegan Restaurant', 'French Restaurant',
       'Fast Food Restaurant', 'Café', 'Japanese Restaurant',
       'Beer Garden', 'Modern European Restaurant', 'Asian Restaurant',
       'Korean Restaurant', 'Indian Restaurant', 'Bar',
       'German Restaurant', 'Thai Restaurant', 'Whisky Bar',
       'Sushi Restaurant', 'Burger Joint', 'Convention Center',
       'Ramen Restaurant', 'Food Court', 'Tapas Restaurant',
       'Middle Eastern Restaurant', 'Breakfast Spot',
       'Chinese Restaurant', 'Hotpot Restaurant', 'Mexican Restaurant'],
      dtype=object)

In [22]:
cafe =['Café', 'Coffee Shop', 'Sandwich Place', 'Cupcake Shop', 'Gourmet Shop','Breakfast Spot']

euro = ['French Restaurant', 'Spanish Restaurant', 'German Restaurant', 
        'Italian Restaurant', 'Pizza Place']

latino = ['Argentinian Restaurant']

bar = ['Beer Bar', 'Cocktail Bar', 'Wine Bar', 'Hotel Bar',
       'Beer Garden','Whisky Bar',  'Pub', 'Bar', 'Juice Bar']

asian = ['Ramen Restaurant', 'Sushi Restaurant', 'Vietnamese Restaurant',
         'Thai Restaurant', 'Indian Restaurant', 
         'Japanese Curry Restaurant', 'Japanese Restaurant', 
         'Noodle House','North Indian Restaurant','Korean Restaurant','Middle Eastern Restaurant', 'Hotpot Restaurant','Asian Restaurant']

american = ['Bistro', 'Burger Joint', 'Fast Food Restaurant',
            'Restaurant', 'BBQ Joint', 
            'Vegetarian / Vegan Restaurant']


def conditions(s):
    if s['categories'] in euro:
        return 'euro'
    if s['categories'] in latino:
        return 'latino'
    if s['categories'] in asian:
        return 'asian'
    if s['categories'] in cafe:
        return 'cafe'
    if s['categories'] in american:
        return 'american'
    if s['categories'] in bar:
        return 'bar'


raw_dataset['categories_classified']=raw_dataset.apply(conditions, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified
3,Pizza 4P,Italian Restaurant,21.025141,105.854438,57d2b9d3498e1e677992b6b5,Hanoi,96,euro
4,Pizza 4P's,Pizza Place,21.029746,105.848956,55deae83498e47aabe8a5e77,Hanoi,246,euro
5,Orchid Cooking Class & Restaurant,Vietnamese Restaurant,21.033874,105.853270,50fbfd43e4b011743042d9d9,Hanoi,80,asian
7,Cộng Càphê,Coffee Shop,21.035166,105.853395,555e9b4a498ea0e936c077a7,Hanoi,87,cafe
8,Starbucks Reserve Nhà Thờ,Coffee Shop,21.029162,105.850222,590d4a5e1108ba582673bec7,Hanoi,44,cafe
9,El Gaucho,Steakhouse,21.024579,105.856475,52974eac498e2e2088c08d4f,Hanoi,74,None
13,Opera Club Novel,Vietnamese Restaurant,21.024660,105.856291,51372fb0e4b05751bf4e3b78,Hanoi,14,asian
14,Nhạc Cafe - Music Cafe,Coffee Shop,21.032465,105.855285,55efd3cb498e9215e3b800f9,Hanoi,290,cafe
15,Dream Beans Coffee,Coffee Shop,21.029088,105.848596,5b4ad8ba6adbf5002c194600,Hanoi,10,cafe
17,Duong's 2 Restaurant & Cooking Class,Vietnamese Restaurant,21.034108,105.853497,5940f170fd16bb5d1e9aedb7,Hanoi,15,asian


In [23]:
# double check to make sure categories_classified has been created correctly

pd.crosstab(index=raw_dataset["categories_classified"],
            columns="count")

col_0,count
categories_classified,
american,16
asian,47
bar,8
cafe,29
euro,14


In [24]:
raw_dataset['likes'] = like_list
raw_dataset.head()

,name,categories,lat,lng,id,city,likes,categories_classified
3,Pizza 4P,Italian Restaurant,21.025141,105.854438,57d2b9d3498e1e677992b6b5,Hanoi,96,euro
4,Pizza 4P's,Pizza Place,21.029746,105.848956,55deae83498e47aabe8a5e77,Hanoi,246,euro
5,Orchid Cooking Class & Restaurant,Vietnamese Restaurant,21.033874,105.853270,50fbfd43e4b011743042d9d9,Hanoi,80,asian
7,Cộng Càphê,Coffee Shop,21.035166,105.853395,555e9b4a498ea0e936c077a7,Hanoi,87,cafe
8,Starbucks Reserve Nhà Thờ,Coffee Shop,21.029162,105.850222,590d4a5e1108ba582673bec7,Hanoi,44,cafe


In [54]:
raw_dataset.sort_values(by=['likes'], ascending=False)

,name,categories,lat,lng,id,city,likes,categories_classified,ranking
10,Pizza 4P's,Pizza Place,10.781866,106.705113,4dd5170c183859a589b15eff,Ho Chi Minh City,514,euro,3
42,Quan An Ngon 18 Phan Boi Chau,Vietnamese Restaurant,21.026405,105.843452,4bab0d1bf964a52024933ae3,Hanoi,455,asian,3
80,Cục Gạch,Vietnamese Restaurant,10.792957,106.689020,4bdfad600ee3a5933d2335b0,Ho Chi Minh City,408,asian,3
19,Bánh Mì 25,Sandwich Place,21.036167,105.848687,54cddaea498ed86c57c8209c,Hanoi,405,cafe,3
24,Propaganda,Vietnamese Restaurant,10.778913,106.697893,52be86e6498e309caaa530fb,Ho Chi Minh City,378,asian,3
40,Bánh Mì Huỳnh Hoa,Sandwich Place,10.771490,106.692380,4cbacc4890c9a143f4f590d6,Ho Chi Minh City,337,cafe,3
72,Phở Hòa Pasteur,Noodle House,10.786644,106.689219,4bff7467daf9c9b65945faef,Ho Chi Minh City,321,asian,3
29,Bún Bò Nam Bộ Hàng Điếu,Noodle House,21.032217,105.846828,4cc91067d147a0935907e5ae,Hanoi,317,asian,3
14,Nhạc Cafe - Music Cafe,Coffee Shop,21.032465,105.855285,55efd3cb498e9215e3b800f9,Hanoi,290,cafe,3
26,The Workshop Cafe,Coffee Shop,10.773551,106.705586,53cc8b78498ec6ddfb2a113d,Ho Chi Minh City,262,cafe,3


In [25]:
# classify the likes into different ranking levels
# lets first see where to bin the data
# we can try different ways of binning the data, I find it yields substantially different results

# print(np.percentile(raw_dataset['likes'], 20))
# print(np.percentile(raw_dataset['likes'], 40))
# print(np.percentile(raw_dataset['likes'], 60))
# print(np.percentile(raw_dataset['likes'], 80))


print(np.percentile(raw_dataset['likes'], 33))
print(np.percentile(raw_dataset['likes'], 66))


# print(np.percentile(raw_dataset['likes'], 50))

22.590000000000003
68.18


In [32]:
# create a function to bin for us

def rankings(s):
    if s['likes']<=22.26:
        return 1
    if s['likes']<=68.52:
        return 2
    if s['likes']>68.52:
        return 3
 

raw_dataset['ranking']=raw_dataset.apply(rankings, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified,ranking
3,Pizza 4P,Italian Restaurant,21.025141,105.854438,57d2b9d3498e1e677992b6b5,Hanoi,96,euro,3
4,Pizza 4P's,Pizza Place,21.029746,105.848956,55deae83498e47aabe8a5e77,Hanoi,246,euro,3
5,Orchid Cooking Class & Restaurant,Vietnamese Restaurant,21.033874,105.853270,50fbfd43e4b011743042d9d9,Hanoi,80,asian,3
7,Cộng Càphê,Coffee Shop,21.035166,105.853395,555e9b4a498ea0e936c077a7,Hanoi,87,cafe,3
8,Starbucks Reserve Nhà Thờ,Coffee Shop,21.029162,105.850222,590d4a5e1108ba582673bec7,Hanoi,44,cafe,2
9,El Gaucho,Steakhouse,21.024579,105.856475,52974eac498e2e2088c08d4f,Hanoi,74,None,3
13,Opera Club Novel,Vietnamese Restaurant,21.024660,105.856291,51372fb0e4b05751bf4e3b78,Hanoi,14,asian,1
14,Nhạc Cafe - Music Cafe,Coffee Shop,21.032465,105.855285,55efd3cb498e9215e3b800f9,Hanoi,290,cafe,3
15,Dream Beans Coffee,Coffee Shop,21.029088,105.848596,5b4ad8ba6adbf5002c194600,Hanoi,10,cafe,1
17,Duong's 2 Restaurant & Cooking Class,Vietnamese Restaurant,21.034108,105.853497,5940f170fd16bb5d1e9aedb7,Hanoi,15,asian,1


In [37]:
# create dummies for linear regression modelling

# one hot encoding
reg_dataset = pd.get_dummies(raw_dataset[['categories_classified', 
                                          'city',]], 
                               prefix="", 
                               prefix_sep="")

# add name, ranking, and likes columns back to dataframe
reg_dataset['ranking'] = raw_dataset['ranking']
reg_dataset['likes'] = raw_dataset['likes']
reg_dataset['name'] = raw_dataset['name']

# move name column to the first column
reg_columns = [reg_dataset.columns[-1]] + list(reg_dataset.columns[:-1])
reg_dataset = reg_dataset[reg_columns]


reg_dataset

,name,american,asian,bar,cafe,euro,Hanoi,Ho Chi Minh City,ranking,likes
3,Pizza 4P,0,0,0,0,1,1,0,3,96
4,Pizza 4P's,0,0,0,0,1,1,0,3,246
5,Orchid Cooking Class & Restaurant,0,1,0,0,0,1,0,3,80
7,Cộng Càphê,0,0,0,1,0,1,0,3,87
8,Starbucks Reserve Nhà Thờ,0,0,0,1,0,1,0,2,44
9,El Gaucho,0,0,0,0,0,1,0,3,74
13,Opera Club Novel,0,1,0,0,0,1,0,1,14
14,Nhạc Cafe - Music Cafe,0,0,0,1,0,1,0,3,290
15,Dream Beans Coffee,0,0,0,1,0,1,0,1,10
17,Duong's 2 Restaurant & Cooking Class,0,1,0,0,0,1,0,1,15


In [39]:
import operator

In [53]:
reg_dataset.sort_values(by=['likes'], ascending=False)

,name,american,asian,bar,cafe,euro,Hanoi,Ho Chi Minh City,ranking,likes
10,Pizza 4P's,0,0,0,0,1,0,1,3,514
42,Quan An Ngon 18 Phan Boi Chau,0,1,0,0,0,1,0,3,455
80,Cục Gạch,0,1,0,0,0,0,1,3,408
19,Bánh Mì 25,0,0,0,1,0,1,0,3,405
24,Propaganda,0,1,0,0,0,0,1,3,378
40,Bánh Mì Huỳnh Hoa,0,0,0,1,0,0,1,3,337
72,Phở Hòa Pasteur,0,1,0,0,0,0,1,3,321
29,Bún Bò Nam Bộ Hàng Điếu,0,1,0,0,0,1,0,3,317
14,Nhạc Cafe - Music Cafe,0,0,0,1,0,1,0,3,290
26,The Workshop Cafe,0,0,0,1,0,0,1,3,262


In [55]:
# draw Hanoi Map 

hochiminh_map = folium.Map(location=[10.8231, 106.6297], zoom_start=12)

# display Hanoi map
hochiminh_map


In [56]:
# instantiate a feature group for the restaurants in the dataframe
restaurants = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(raw_dataset.lat, raw_dataset.lng):
    restaurants.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(raw_dataset.lat)
longitudes = list(raw_dataset.lng)

for lat, lng in zip(latitudes, longitudes):
    folium.Marker([lat, lng]).add_to(hochiminh_map)    
    
# add incidents to map
hochiminh_map.add_child(restaurants)